In [ ]:
!pip install onnxruntime-genai
!pip install onnxruntime
!pip install olive
!pip install huggingface-hub

In [ ]:
!huggingface-cli login --token <TOKEN>

In [1]:
import olive.workflows

olive.workflows.run("olive.json")

[2024-04-11 10:48:54,807] [INFO] [run.py:243:run] Loading Olive module configuration from: C:\Users\nakersha\AppData\Local\miniconda3\envs\python3.11\Lib\site-packages\olive\olive_config.json
[2024-04-11 10:48:54,807] [INFO] [run.py:249:run] Loading run configuration from: olive.json
[2024-04-11 10:48:54,816] [INFO] [config.py:184:validate_evaluate_input_model] No evaluator is specified, skip to evaluate model
[2024-04-11 10:48:54,866] [DEBUG] [accelerator.py:245:normalize_accelerators] The accelerator device and execution providers are specified, skipping deduce.
[2024-04-11 10:48:54,867] [DEBUG] [accelerator.py:275:normalize_accelerators] Supported execution providers for device cpu: ['CPUExecutionProvider']
[2024-04-11 10:48:54,868] [DEBUG] [accelerator.py:302:create_accelerators] Initial accelerators and execution providers: {'CPU': ['CPUExecutionProvider']}
[2024-04-11 10:48:54,868] [INFO] [accelerator.py:324:create_accelerators] Running workflow on accelerator specs: cpu-cpu
[202

{AcceleratorSpec(accelerator_type=<Device.CPU: 'cpu'>, execution_provider='CPUExecutionProvider', vender=None, version=None, memory=None, num_cores=None): <olive.engine.footprint.Footprint at 0x1ec4cf12610>}

In [4]:
import onnxruntime_genai as og
import time

print("Loading model...")
app_started_timestamp = time.time()

model = og.Model(f'example-models/microsoft/phi-2/cpu/int4/genai_exporter/cpu-cpu_model')
model_loaded_timestamp  = time.time()

print("Model loaded in {:.2f} seconds".format(model_loaded_timestamp - app_started_timestamp))



Loading model...
Model loaded in 23.99 seconds


In [5]:
print("Loading tokenizer...")
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

print("Tokenizer created")

system_prompt = "You are a helpful assistant. Answer in one sentence."
text = "What is Dilithium?"

input_tokens = tokenizer.encode(system_prompt + text)

prompt_length = len(input_tokens)

Loading tokenizer...
Tokenizer created


In [6]:
started_timestamp = time.time()

print("Creating generator ...")
params = og.GeneratorParams(model)
params.set_search_options({"do_sample": False, "max_length": 2028, "min_length": 0, "top_p": 0.9, "top_k": 40, "temperature": 1.0, "repetition_penalty": 1.0})
params.input_ids = input_tokens
generator = og.Generator(model, params)
print("Generator created")

first = True
new_tokens = []

while not generator.is_done():
    generator.compute_logits()
    generator.generate_next_token()
    if first:
        first_token_timestamp = time.time()
        first = False

    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end="")
    new_tokens.append(new_token)

print()
run_time = time.time() - started_timestamp
print(f"Prompt tokens: {len(input_tokens)}, New tokens: {len(new_tokens)}, Time to first: {(first_token_timestamp - started_timestamp):.2f}s, New tokens per second: {len(new_tokens)/run_time:.2f} tps")


Creating generator ...
Generator created

A: Dilithium is a fictional substance in the Star Trek universe that is used as a propellant and a power source for spaceships.

Prompt tokens: 17, New tokens: 32, Time to first: 0.62s, New tokens per second: 10.50 tps


## Download and build llama.cpp
* git clone https://github.com/ggerganov/llama.cpp.git
* cd llama.cpp
* cmake -S . -B build/ -D CMAKE_BUILD_TYPE=Release
* cmake --build build/ --config Release

## Download gguf phi-2 model
* cd ..
* git clone https://huggingface.co/TheBloke/phi-2-GGUF

In [9]:
# Compare with llama.cpp.

! ..\..\ggerganov\llama.cpp\build\bin\Release\main -m ..\..\thebloke\phi-2-GGUF\phi-2.Q4_K_M.gguf --prompt "You are a helpful assistant. Answer in one sentence. What is Dilithium?"


You are a helpful assistant. Answer in one sentence. What is Dilithium? OUTPUT: Dilithium is a fictional element from the Star Trek universe that is used as a powerful fuel for space travel.


Log start
main: build = 2647 (8228b66d)
main: built with MSVC 19.39.33523.0 for x64
main: seed  = 1712858073
llama_model_loader: loaded meta data with 20 key-value pairs and 325 tensors from ..\..\thebloke\phi-2-GGUF\phi-2.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi2
llama_model_loader: - kv   1:                               general.name str              = Phi2
llama_model_loader: - kv   2:                        phi2.context_length u32              = 2048
llama_model_loader: - kv   3:                      phi2.embedding_length u32              = 2560
llama_model_loader: - kv   4:                   phi2.feed_forward_length u32              = 10240
llama_model_loader: - kv   5:                           phi2.block_count u32              = 32
llama_model_loader: - kv   6:                  phi2.attenti